In [169]:
# !pip install cassandra-driver

In [172]:
# from cassandra.cluster import Cluster
# cluster = Cluster(['127.0.0.1'])
# session = cluster.connect()

In [178]:
# session.execute("use dev")
# session.execute("CREATE TABLE users (id int PRIMARY KEY, address text)")

In [ ]:
import os
os.chdir("..")
os.chdir("..")

In [2]:
os.environ["PYSPARK_SUBMIT_ARGS"] = "--packages com.datastax.spark:spark-cassandra-connector_2.11:2.5.0 --conf spark.cassandra.connection.host=127.0.0.1 pyspark-shell"

In [14]:
base_path = os.getcwd()

In [15]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql import Window


In [16]:
files_path = os.path.join(base_path, "00_input", "data")

In [17]:
spark : SparkSession = SparkSession.builder\
    .master( "local[*]" ) \
    .appName( "Cassandra-spark-example" ) \
    .config("spark.cassandra.connection.host", "127.0.0.1") \
    .getOrCreate()

In [19]:
data.columns

['countriesAndTerritories',
 'location',
 'continent',
 'population_year',
 'population',
 'id']

In [20]:
cols = ['id', 'location', 'continent', 'population_year', 'population'] #'countriesAndTerritories',
data.select(*cols).write\
    .format("org.apache.spark.sql.cassandra")\
    .mode('append')\
    .options(table="populations", keyspace="dev")\
    .save()

In [23]:
cass_data = spark.read\
    .format("org.apache.spark.sql.cassandra")\
    .options(table="populations", keyspace="dev")\
    .load()

In [24]:
cass_data.printSchema()

root
 |-- id: integer (nullable = false)
 |-- continent: string (nullable = true)
 |-- location: string (nullable = true)
 |-- population: integer (nullable = true)
 |-- population_year: integer (nullable = true)



In [25]:
cass_data.count()

209

In [20]:
data = spark\
    .read\
    .options(header='true', inferSchema='true')\
    .csv(f"file://{files_path}/locations.csv")\
    .withColumn("id", monotonically_increasing_id().astype("int"))

In [26]:
cass_data.groupBy("continent").count().show()

+-------------+-----+
|    continent|count|
+-------------+-----+
|       Europe|   51|
|       Africa|   54|
|         null|    1|
|North America|   36|
|South America|   13|
|      Oceania|    8|
|         Asia|   46|
+-------------+-----+



In [50]:
window = Window.partitionBy("continent")

In [55]:
max_df = cass_data.withColumn("max_population", max("population").over(window))\
  .filter(col("max_population") == col("population")).orderBy("continent")

In [56]:
max_df.show()

+---+-------------+-------------+----------+---------------+--------------+
| id|    continent|     location|population|population_year|max_population|
+---+-------------+-------------+----------+---------------+--------------+
|141|       Africa|      Nigeria| 206139587|           2020|     206139587|
| 42|         Asia|        China|1439323774|           2020|    1439323774|
|158|       Europe|       Russia| 145934460|           2020|     145934460|
|200|North America|United States| 331002647|           2020|     331002647|
| 10|      Oceania|    Australia|  25499881|           2020|      25499881|
| 27|South America|       Brazil| 212559409|           2020|     212559409|
+---+-------------+-------------+----------+---------------+--------------+

